In [1]:
import os
import numpy as np
import scipy.io as sio
import glob
import torch
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
from torchvision.models.vgg import vgg16
from torchvision.models.alexnet import alexnet

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
transform  = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
def main():

  alex_feature = []
  alex_label = []

  vgg16_feature = []
  vgg16_label = []

  train_data = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

  # test_data = torchvision.datasets.CIFAR10(root='./data', train=False,
  #                                       download=True, transform=transform)


  # [Problem 4 a.] IMPORT VGG16 AND ALEXNET FROM THE MODELS FOLDER WITH
  # PRETRAINED = TRUE


  vgg16_extractor = vgg16(pretrained=True).to(device)
  # vgg16_extractor = vgg16(pretrained=True)
  vgg16_extractor.eval()

  # alex_extractor = alexnet(pretrained=True)
  alex_extractor = alexnet(pretrained=True).to(device)
  alex_extractor.eval()


  for idx, data in enumerate(train_data):

      image, label = data

      image = image.to(device)

      #Forward pass through VGG16
      with torch.no_grad():
          F_vgg = vgg16_extractor(image.unsqueeze(0))
          vgg16_feature.append(F_vgg.flatten().cpu().numpy())  # Move back to CPU
          vgg16_label.append(label)

      # Forward pass through AlexNet
      with torch.no_grad():
          F_alex = alex_extractor(image.unsqueeze(0))
          alex_feature.append(F_alex.flatten().cpu().numpy())  # Move back to CPU
          alex_label.append(label)


  sio.savemat('vgg16.mat', mdict={'feature': alex_feature, 'label': alex_label})
  sio.savemat('alexnet.mat', mdict={'feature': vgg16_feature, 'label': vgg16_label})

In [5]:
def KNN_test(test_data, K=1):
    # Load the saved .mat files
    vgg_mat = sio.loadmat('vgg16.mat')['feature']
    alex_mat = sio.loadmat('alexnet.mat')['feature']
    alex_labels = sio.loadmat('alexnet.mat')['label'].flatten()

    # Load models and move to GPU if available
    vgg16_extractor = vgg16(pretrained=True).to(device)
    alex_extractor = alexnet(pretrained=True).to(device)

    vgg16_accuracy = 0
    alex_accuracy = 0

    for idx, (data, label) in enumerate(test_data):
        if idx == 300:
            break
        # 1. EXTRACT FEATURES USING THE MODELS - ALEXNET AND VGG16
        data = data.unsqueeze(0).to(device)  # Add batch dimension and move to GPU
        with torch.no_grad():
            F_test_vgg16 = vgg16_extractor(data).flatten().cpu().numpy()
            F_test_alex = alex_extractor(data).flatten().cpu().numpy()

        # 2. FIND NEAREST NEIGHBOR OF THIS FEATURE FROM FEATURES STORED IN ALEXNET.MAT AND VGG16.MAT
        nbrs_vgg = NearestNeighbors(n_neighbors=K, algorithm='ball_tree').fit(vgg_mat)
        distances_vgg, indices_vgg = nbrs_vgg.kneighbors([F_test_vgg16])

        nbrs_alex = NearestNeighbors(n_neighbors=K, algorithm='ball_tree').fit(alex_mat)
        distances_alex, indices_alex = nbrs_alex.kneighbors([F_test_alex])

        # 3. COMPUTE ACCURACY
        # The labels are the K-nearest neighbor labels, not indices, so we need to extract them
        vgg16_nearest_labels = [alex_labels[ind] for ind in indices_vgg[0]]
        alex_nearest_labels = [alex_labels[ind] for ind in indices_alex[0]]

        vgg16_accuracy += accuracy_score([label], vgg16_nearest_labels)
        alex_accuracy += accuracy_score([label], alex_nearest_labels)

    vgg16_accuracy /= len(test_data)
    alex_accuracy /= len(test_data)

    return vgg16_accuracy, alex_accuracy

In [ ]:
main()

Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  w

In [8]:
test_data = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

# Test using KNN
vgg16_acc, alex_acc = KNN_test(test_data)
print("VGG16 Accuracy:", vgg16_acc)
print("AlexNet Accuracy:", alex_acc)


VGG16 Accuracy: 0.52
AlexNet Accuracy: 0.46
